In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

  ERROR: Error [Errno 2] No such file or directory: 'git' while executing command git clone -q https://github.com/huggingface/peft.git /tmp/pip-req-build-quxhkppy
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [2]:
!pip install peft transformers

     |████████████████████████████████| 56 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 772 kB 29.3 MB/s eta 0:00:01


In [4]:
!pip install scipy

     |████████████████████████████████| 34.5 MB 881 kB/s eta 0:00:01


# Confirming CUDA

# Loading Base model

In [27]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1", 
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.02s/it]


# Model Summary

In [28]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear8bitLt(in_features=4096, out_features=12288, bias=True)
          (dense): Linear8bitLt(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear8bitLt(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, eleme

# Freezing the layers for PEFT and LORA

In [29]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [30]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# LoRA config

In [31]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3932160 || all params: 7072948224 || trainable%: 0.055594355783029126


# Loading the data

In [32]:
from datasets import load_dataset

dataset = load_dataset("Jayveersinh-Raj/bad-improved-prompt-pairs")

Found cached dataset csv (/root/.cache/huggingface/datasets/Jayveersinh-Raj___csv/Jayveersinh-Raj--bad-improved-prompt-pairs-3035872143f22a11/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 234.75it/s]


In [33]:
dataset

DatasetDict({
    train: Dataset({
        features: ['bad_prompts', 'improved_prompts'],
        num_rows: 1217
    })
})

In [48]:
dataset["train"][0]

{'bad_prompts': 'Write a program that calculates the area of a triangle.',
 'improved_prompts': ' "Implement a Python program that takes the lengths of the sides of a triangle as inputs and calculates its area using Heron\'s formula"'}

# Prompt tuning

In [69]:
def create_prompt(prompt, improved):
  # If no improvement pass the original prompt
  if improved == None:
    answer = prompt
  else:
    answer = improved
  prompt_template = f"### Bad prompt\n{prompt}\n\n### Improved/tuned prompt\n{answer}</s>"
  return prompt_template

mapped_dataset = dataset.map(lambda samples: tokenizer(create_prompt(samples['bad_prompts'], samples['improved_prompts'])))

In [70]:
mapped_dataset

DatasetDict({
    train: Dataset({
        features: ['bad_prompts', 'improved_prompts', 'input_ids', 'attention_mask'],
        num_rows: 1217
    })
})

In [71]:
mapped_dataset["train"][0]

{'bad_prompts': 'Write a program that calculates the area of a triangle.',
 'improved_prompts': ' "Implement a Python program that takes the lengths of the sides of a triangle as inputs and calculates its area using Heron\'s formula"',
 'input_ids': [105311,
  34094,
  39841,
  189,
  22687,
  267,
  3692,
  861,
  223175,
  368,
  10583,
  461,
  267,
  57890,
  6149,
  105311,
  127185,
  6410,
  29733,
  309,
  376,
  39841,
  189,
  29000,
  4146,
  1689,
  267,
  29788,
  3692,
  861,
  18456,
  368,
  119623,
  461,
  368,
  42425,
  461,
  267,
  57890,
  661,
  59499,
  530,
  223175,
  3776,
  10583,
  3936,
  14367,
  170379,
  25446,
  5,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

# Training

In [83]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        #max_steps=10,
        num_train_epochs = 5,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=100,
        save_steps=100,
        overwrite_output_dir=True,
        save_total_limit=3,
        output_dir='bloom-prompt-tune',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
100,0.372300
200,0.423200
300,0.332700


/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=380, training_loss=0.35109990772448085, metrics={'train_runtime': 1144.8543, 'train_samples_per_second': 5.315, 'train_steps_per_second': 0.332, 'total_flos': 9785876004225024.0, 'train_loss': 0.35109990772448085, 'epoch': 4.98})

# Checking

In [84]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "bloom-prompt-tune/checkpoint-300"
config = PeftConfig.from_pretrained(peft_model_id)
model_loaded = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
qa_model = PeftModel.from_pretrained(model_loaded, peft_model_id)

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.60s/it]


In [75]:
from IPython.display import display, Markdown

def make_inference(prompt):
  batch = tokenizer(f"### Bad prompt\n{prompt}\n\n### Improved/tuned prompt\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [87]:
prompt = "write a djitkra algorithm in python please also I want it optimized you know"
make_inference(prompt)

### Bad prompt
write a djitkra algorithm in python please also I want it optimized you know

### Improved/tuned prompt
Write a djikstra algorithm in python, which calculates the shortest path between any two nodes in a graph.